### resource limits
Indicates that the output limit seems to be based on size.  
https://docs.heygen.com/reference/limits#resource-limits

### streaming generation (real-time interactive gen)
It appears there is a beta version that allows interactive generation as well (in addition to async).  
https://docs.heygen.com/reference/overview-copy#video-tutorials

In [1]:
import requests
import json
import os

In [2]:
# get from env var
HG_API_KEY = os.environ["HG_API_KEY"]

### list avatars already present

In [6]:
URL_AVATAR_LIST = "https://api.heygen.com/v2/avatars"

headers = {
    'accept': 'application/json',
    'x-api-key': HG_API_KEY 
}


In [7]:
avatar_list_response = requests.get(url=URL_AVATAR_LIST, headers=headers)

In [18]:
print(json.dumps(avatar_list_response.json()["data"]["talking_photos"][:2], indent=4))

[
    {
        "talking_photo_id": "aacd116712c6487b8db4bfe40243c0e0",
        "talking_photo_name": "TalkingPhoto",
        "preview_image_url": "https://files.heygen.ai/prod/movio/url_upload/user_upload/e664822f52e742e09e517d16ed85811b/c255e3c4db0c4925806311bc6cf8ffbb.png?Expires=1706903562&Signature=TmMq6NXH6Fd~QairekonjY3L-5cAXnAAwtwLZjVgDhy~Qeh9cWNWiWyBVSj-4j6KyWNQ4s2oXzbqEp3PCkboAhnnsVA-RjXrCcBpKhmozA7dm4pH2gYvHZWWNgIN0U7Fa4~XvoVqz9tmRQYtvaoHrPNLJaEtZ2uHfrA~xKtVHYPh1vRlEf0ejQsW9gJAICZR-rktHFLcJNRoNHpR99tK4V91p77tpbTi~Oodp8-1ydYpmoEkMqE2lcZNvBnZcDADOSvWCJvJFVIpAbLJDmwd5C1KW6H235t-KozWB9jwxxHh6M~eEldfw99WDiUucYGvVhjDvqRT4oOEpcmkw211xQ__&Key-Pair-Id=K49TZTO9GZI6K"
    },
    {
        "talking_photo_id": "ba9c11684315405aac1dd8ed987fdda2",
        "talking_photo_name": "Felix",
        "preview_image_url": "https://files.heygen.ai/prod/movio/url_upload/user_upload/632e0fe43dbf472cabecf45d5780856f/35dcedc4cc4e4d7db61b341e9c2aa271.jpeg?Expires=1707010866&Signature=dd038opJ9KaA2P5ohvb

### generate video

Among many input fields, the three important ones are the following:  
* input_text - used for text-to-speech (TTS) content
* talking_photo_id (or avatar or videos ids) - reference image/video identifiers in the account. I have used a static image as talking_photo in this example.
* voice_id - audio reference for generated TTS content.

### async behavior
* The POST call to `/v2/video/generate` will give identifiers in its response for subsequent status check calls to `video_status.get`
* `/video_status.get` the job status and generated video URLs

So, a wrapper module/API around these would follow a similar logic

In [76]:
VIDEO_GENERATE_URL = "https://api.heygen.com/v2/video/generate"

In [5]:
# ID of an image already uploaded to heygen account
# heygen apparently categorizes it as "talking photo"
talking_photo_id = "aacd116712c6487b8db4bfe40243c0e0"

# input prompt for TTS
input_text = """
After the Japan Aerospace Exploration Agency (Jaxa) re-established contact with its Moon lander on Sunday night, the craft started to study its surroundings in detail and transmit new images to Earth.
The lander is carrying out analysis that could inform future, much longer missions to the Moon.
One lunar scientist told BBC News: "Now we're really seeing the science."

The craft is at a very awkward angle. A picture, captured by the small baseball-sized robot called Sora-Q - which was ejected from Slim moments before touchdown - showed the lander face-down on the lunar surface.
That left its solar panels facing away from the sunlight and unable to generate power. The decision was taken to put the lander into sleep mode - and conserve what power remained - less than three hours after it landed.

"""

voice_id = "d7bbcdd6964c47bdaae26decade4a933"

In [78]:
payload = {
    "video_inputs": [
        {
            "character": {
                "type": "talking_photo",
                "talking_photo_id": talking_photo_id
            },
            "voice": {
                "type": "text",
                "input_text": input_text,
                "voice_id": voice_id
            },
            "background": {
                "type": "color",
                "value": "#FAFAFA"
            }
        }
    ],
  "test": True,
  "aspect_ratio": "16:9",
  "version": "v1alpha"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    'x-api-key': HG_API_KEY 
}


In [79]:
response = requests.post(url=VIDEO_GENERATE_URL, json=payload, headers=headers)

In [80]:
print(response.text)

{"data":null,"error":{"code":"trial_video_limit_exceeded","message":"It seems you've reached your daily api trial limit 5."}}



In [62]:
# video_id = "2cd544e8fe5c4b0b94a630d4e1583265"
video_id = "91c671fefa4b4273b570793fb976b281"

In [63]:
# check status of job
BASE_VIDEO_STATUS_URL = "https://api.heygen.com/v1/video_status.get"
video_status_url = BASE_VIDEO_STATUS_URL+"?video_id="+video_id
print(video_status_url)
status_headers = {
    "accept": "application/json",
    'x-api-key': HG_API_KEY 
}

status_response = requests.get(url=video_status_url, json=payload, headers=status_headers)

https://api.heygen.com/v1/video_status.get?video_id=91c671fefa4b4273b570793fb976b281


In [64]:
print(status_response.text)

{"code":100,"data":{"callback_id":null,"caption_url":"","duration":1.17549e-38,"error":null,"gif_url":null,"id":"91c671fefa4b4273b570793fb976b281","status":"completed","thumbnail_url":"https://files.heygen.ai/aws_pacific/avatar_tmp/e664822f52e742e09e517d16ed85811b/ba69396d83914fb18398f0fa5e2f5b19.jpg?Expires=1707157634&Signature=IDZt7L34eHpWbYQIRPYBktEQNxmlUPo2gKxqsOXtd0quPq61Jek1WFOusvQxHuzfLqzqIIKaNFssatWX7VPL0HwaICwTg1LYSc8Ba-n~rFwRsNTBeQBVJsVWkFvk7vjytHnOQnFyi3M7YdsmIPTY~tUX22QvsUquSIn9LLRBa7RRISaxh5XxEMNTHEmQv1fxIGREOH9iSE75mk68IdSeodwY5OZ78101a21F6YRlZ1wmmFtUkaNghAxHPCVUxSCSE2gcVQzUUN3LRbsIBiZ1x6uLYiXSRC5684UlcSSM5yYiO8x52Zj71BPCtrjkA38XE1-rRJCTn0jBSv57tq6GMw__&Key-Pair-Id=K49TZTO9GZI6K","video_url":"https://files.heygen.ai/aws_pacific/avatar_tmp/e664822f52e742e09e517d16ed85811b/bdedca9878494f4e844b500aaf724920.mp4?Expires=1707157634&Signature=R3-n3QzIgNMTdap9Kb1wBa3CIV-yid8Ce3ee7pteNrGBQOkBs8RvRS2NM-Gmrra6rIL4Cz18wkdZalsCFw7-Kw6e-KAK0DvB3MfZ4gOnY-yeibGfWlVM8I8skYEUpbyi5-O42OywiW